In [8]:
import pandas as pd
import numpy as np

In [9]:

# Read the Verbraucher Preisindex CSV
# Read the CSV with semicolon separator and specify encoding if needed
df_vpi = pd.read_csv("Verbraucherpreisindex.csv", sep=';', header=None, 
                        names=['Jahr', 'Monat', 'VPI'], 
                        encoding='utf-8')  # Adjust encoding if needed



df_vpi

,Jahr,Monat,VPI
0,2013,Januar,"97,4"
1,2013,Februar,"98,0"
2,2013,März,"98,4"
3,2013,April,"98,0"
4,2013,Mai,"98,4"
...,...,...,...
91,2020,August,"106,0"
92,2020,September,"105,8"
93,2020,Oktober,"105,9"
94,2020,November,"105,0"


In [10]:
print(df_vpi.head())

   Jahr    Monat   VPI
0  2013   Januar  97,4
1  2013  Februar  98,0
2  2013     März  98,4
3  2013    April  98,0
4  2013      Mai  98,4


In [11]:
# Create a mapping of German month names to numbers
month_mapping = {
    'Januar': 1, 'Februar': 2, 'März': 3, 'April': 4, 
    'Mai': 5, 'Juni': 6, 'Juli': 7, 'August': 8, 
    'September': 9, 'Oktober': 10, 'November': 11, 'Dezember': 12
}

df_vpi['Monat'] = df_vpi['Monat'].map(month_mapping)

df_vpi.head()


# # Convert month names to numbers
# df_vpi['Monat_Nummer'] = df_vpi['Monat'].map(month_mapping)

# # Create a datetime column
# df_vpi['Datum'] = pd.to_datetime(
#     df_vpi['Jahr'].astype(str) + '-' + 
#     df_vpi['Monat_Nummer'].astype(str) + 
#     '-01'
# )

,Jahr,Monat,VPI
0,2013,1,"97,4"
1,2013,2,"98,0"
2,2013,3,"98,4"
3,2013,4,"98,0"
4,2013,5,"98,4"


In [12]:
df_vpi['VPI'] = df_vpi['VPI'].str.replace(',', '.')
df_vpi['VPI'] = df_vpi['VPI'].astype(float)
df_vpi.head()

,Jahr,Monat,VPI
0,2013,1,97.4
1,2013,2,98.0
2,2013,3,98.4
3,2013,4,98.0
4,2013,5,98.4


In [13]:
df_vpi['Datum'] = pd.to_datetime(df_vpi['Jahr'].astype(str) + '-' + df_vpi['Monat'].astype(str) + '-01')
df_vpi.head()

,Jahr,Monat,VPI,Datum
0,2013,1,97.4,2013-01-01
1,2013,2,98.0,2013-02-01
2,2013,3,98.4,2013-03-01
3,2013,4,98.0,2013-04-01
4,2013,5,98.4,2013-05-01


In [14]:
# Keep only Datum and VPI
df_vpi = df_vpi[['Datum', 'VPI']]
df_vpi

,Datum,VPI
0,2013-01-01,97.4
1,2013-02-01,98.0
2,2013-03-01,98.4
3,2013-04-01,98.0
4,2013-05-01,98.4
...,...,...
91,2020-08-01,106.0
92,2020-09-01,105.8
93,2020-10-01,105.9
94,2020-11-01,105.0


In [16]:
# Create a date range for all days
date_range = pd.date_range(start='2013-07-01', end='2018-07-31', freq='D')

# Create a new DataFrame with this date range
df_vpi_daily = pd.DataFrame({'Datum': date_range})

# Merge with original VPI data, using the month start as the key
df_vpi_daily['year_month'] = df_vpi_daily['Datum'].dt.to_period('M')
df_vpi['year_month'] = df_vpi['Datum'].dt.to_period('M')

# Merge and cleanup
df_vpi_daily = df_vpi_daily.merge(df_vpi[['year_month', 'VPI']], on='year_month')
df_vpi_daily = df_vpi_daily[['Datum', 'VPI']]

df_vpi_daily['VPI'] = df_vpi_daily['VPI'] / 106.2
df_vpi_daily

/tmp/ipykernel_8756/4036961212.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vpi['year_month'] = df_vpi['Datum'].dt.to_period('M')


,Datum,VPI
0,2013-07-01,0.931262
1,2013-07-02,0.931262
2,2013-07-03,0.931262
3,2013-07-04,0.931262
4,2013-07-05,0.931262
...,...,...
1852,2018-07-27,0.983051
1853,2018-07-28,0.983051
1854,2018-07-29,0.983051
1855,2018-07-30,0.983051
